# Ehita regressioonimudel: valmista ette ja visualiseeri andmed

## **Lineaarne regressioon kõrvitsate jaoks - Õppetund 2**
#### Sissejuhatus

Nüüd, kui sul on olemas tööriistad, et alustada masinõppe mudelite loomist Tidymodelsi ja Tidyverse'i abil, oled valmis hakkama oma andmetele küsimusi esitama. Andmetega töötades ja ML-lahendusi rakendades on väga oluline osata esitada õigeid küsimusi, et täielikult avada oma andmekogumi potentsiaal.

Selles õppetunnis õpid:

-   Kuidas valmistada andmeid ette mudeli loomiseks.

-   Kuidas kasutada `ggplot2` andmete visualiseerimiseks.

Küsimus, millele vastust otsid, määrab, millist tüüpi ML-algoritme sa kasutad. Ja saadud vastuse kvaliteet sõltub suuresti sinu andmete olemusest.

Vaatame seda praktilise harjutuse kaudu.


<p >
   <img src="../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.et.jpg"
   width="700"/>
   <figcaption>Kunstiteos: @allison_horst</figcaption>


<!--![Kunstiteos: \@allison_horst](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.et.jpg)<br>Kunstiteos: \@allison_horst-->


## 1. Kõrvitsate andmete importimine ja Tidyverse'i kasutuselevõtt

Selle õppetüki andmete töötlemiseks vajame järgmisi pakette:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) on [R-pakettide kogum](https://www.tidyverse.org/packages), mis on loodud selleks, et muuta andmeteadus kiiremaks, lihtsamaks ja lõbusamaks!

Saate need installida järgmiselt:

`install.packages(c("tidyverse"))`

Allolev skript kontrollib, kas teil on selle mooduli läbimiseks vajalikud paketid olemas, ja installib need, kui mõni neist puudub.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Nüüd käivitame mõned paketid ja laadime selle õppetunni jaoks ette nähtud [andmed](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv)!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Kiire `glimpse()` näitab kohe, et andmetes on tühikud ning segamini stringid (`chr`) ja numbrilised andmed (`dbl`). `Date` on tüüpi karakter ning on ka kummaline veerg nimega `Package`, kus andmed on segu `sacks`, `bins` ja muudest väärtustest. Tegelikult on andmed üsna segased 😤.

Tõepoolest, pole väga tavaline saada andmekogumit, mis oleks täiesti valmis ML-mudeli loomiseks otse karbist välja. Aga ära muretse, selles õppetükis õpid, kuidas ette valmistada toorest andmekogumit, kasutades standardseid R-i teeke 🧑‍🔧. Samuti õpid erinevaid tehnikaid andmete visualiseerimiseks.📈📊
<br>

> Meeldetuletus: Pipe-operaator (`%>%`) teostab operatsioone loogilises järjestuses, edastades objekti edasi funktsiooni või väljendisse. Võid mõelda pipe-operaatorist kui koodis "ja siis" ütlemisest.


## 2. Kontrolli puuduvate andmete olemasolu

Üks levinumaid probleeme, millega andmeteadlased peavad tegelema, on puudulikud või puuduvad andmed. R tähistab puuduvaid või teadmata väärtusi spetsiaalse märgiga: `NA` (Not Available).

Kuidas siis teada saada, et andmeraamistik sisaldab puuduvaid väärtusi?
<br>
-   Üks lihtne viis oleks kasutada R-i baasfunktsiooni `anyNA`, mis tagastab loogilised objektid `TRUE` või `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Suurepärane, tundub, et mõned andmed on puudu! See on hea koht alustamiseks.

-   Teine võimalus oleks kasutada funktsiooni `is.na()`, mis näitab, millised üksikud veeru elemendid on puudu, tagastades loogilise väärtuse `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Okei, töö sai tehtud, kuid nii suure andmeraamiga nagu see, oleks ebaefektiivne ja praktiliselt võimatu kõiki ridu ja veerge individuaalselt üle vaadata😴.

- Intuitiivsem viis oleks arvutada puuduvate väärtuste summa iga veeru kohta:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Palju parem! Mõned andmed on puudu, kuid võib-olla see ei ole antud ülesande jaoks oluline. Vaatame, mida edasine analüüs toob.

> Lisaks suurepärastele pakettidele ja funktsioonidele on R-il väga hea dokumentatsioon. Näiteks kasuta `help(colSums)` või `?colSums`, et saada rohkem teavet funktsiooni kohta.


## 3. Dplyr: Andmete manipuleerimise grammatika

<p>
   <img src="../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.et.png"
   width="569"/>
   <figcaption>Kunstiteos: @allison_horst</figcaption>


[`dplyr`](https://dplyr.tidyverse.org/), Tidyverse'i pakett, on andmetöötluse grammatika, mis pakub ühtset komplekti tegusõnu, mis aitavad lahendada kõige tavalisemaid andmetöötluse väljakutseid. Selles jaotises uurime mõningaid dplyr-i tegusõnu!  
<br>


#### dplyr::select()

`select()` on funktsioon paketis `dplyr`, mis aitab valida veerge, mida säilitada või välja jätta.

Et muuta oma andmeraamistikku lihtsamini hallatavaks, eemalda mitmed veerud, kasutades `select()` ja säilitades ainult vajalikud veerud.

Näiteks selles harjutuses hõlmab meie analüüs veerge `Package`, `Low Price`, `High Price` ja `Date`. Valime need veerud.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` on funktsioon paketis `dplyr`, mis aitab luua või muuta veerge, säilitades olemasolevad veerud.

`mutate` üldine struktuur on:

`data %>%   mutate(new_column_name = what_it_contains)`

Vaatame, kuidas `mutate` töötab, kasutades `Date` veergu ja tehes järgmised toimingud:

1. Muudame kuupäevad (praegu tüüpi character) kuu formaadiks (need on USA kuupäevad, seega formaat on `MM/DD/YYYY`).

2. Ekstraheerime kuupäevadest kuu uude veergu.

R-is teeb pakett [lubridate](https://lubridate.tidyverse.org/) kuupäeva- ja ajandmetega töötamise lihtsamaks. Seega kasutame `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` ja vaatame, kuidas ülaltoodud eesmärke saavutada. Võime `Date` veeru eemaldada, kuna me ei vaja seda edasistes toimingutes.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Hurraa! 🤩

Järgmisena loome uue veeru `Price`, mis tähistab kõrvitsa keskmist hinda. Nüüd arvutame veergude `Low Price` ja `High Price` keskmise, et täita uus veerg Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Jah!💪

"Aga oota!", ütled sa pärast kogu andmestiku kiiret ülevaatamist `View(pumpkins)` abil, "Siin on midagi kummalist!"🤔

Kui vaatad `Package` veergu, müüakse kõrvitsaid mitmes erinevas konfiguratsioonis. Mõned müüakse `1 1/9 bushel` mõõtühikus, mõned `1/2 bushel` mõõtühikus, mõned tüki kaupa, mõned naela kaupa ja mõned suurtes kastides, mille laius varieerub.

Kontrollime seda:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Hämmastav!👏

Tundub, et kõrvitsaid on väga raske järjepidevalt kaaluda, seega filtreerime need, valides ainult kõrvitsad, mille `Package` veerus on string *bushel*, ja paneme need uude andmeraami `new_pumpkins`.


#### dplyr::filter() ja stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): loob andmehulga alamhulga, mis sisaldab ainult **ridasi**, mis vastavad teie tingimustele, antud juhul kõrvitsaid, mille `Package` veerus on string *bushel*.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): tuvastab mustri olemasolu või puudumise stringis.

[`stringr`](https://github.com/tidyverse/stringr) pakett pakub lihtsaid funktsioone levinud stringitoimingute jaoks.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Näete, et oleme kitsendanud andmed umbes 415 reani, mis sisaldavad kõrvitsaid korviga.🤩
<br>


#### dplyr::case_when()

**Aga oota! On veel üks asi, mida teha**

Kas märkasid, et busheli kogus varieerub ridade kaupa? Pead hindade normaliseerimiseks näitama hinda ühe busheli kohta, mitte 1 1/9 või 1/2 busheli kohta. On aeg teha veidi matemaatikat, et seda standardiseerida.

Kasutame funktsiooni [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html), et *muuta* Price veergu vastavalt teatud tingimustele. `case_when` võimaldab vektoriseerida mitmeid `if_else()` avaldusi.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Nüüd saame analüüsida ühiku hinda, lähtudes nende mõõtmisest bushelites. Kogu see kõrvitsate bushelite uurimine näitab aga, kui `oluline` on `mõista oma andmete olemust`!

> ✅ Vastavalt [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) andmetele sõltub busheli kaal toodete tüübist, kuna see on mahu mõõtühik. "Näiteks peaks tomatite bushel kaaluma 56 naela... Lehed ja rohelised võtavad rohkem ruumi, kuid kaaluvad vähem, seega kaalub spinati bushel ainult 20 naela." See kõik on üsna keeruline! Ärgem vaevugem busheli ja naela vahelise teisendusega ning määrame hinna busheli järgi. Kogu see kõrvitsate bushelite uurimine näitab aga, kui oluline on mõista oma andmete olemust!
>
> ✅ Kas märkasid, et poolte bushelite kaupa müüdavad kõrvitsad on väga kallid? Kas oskad arvata, miks? Vihje: väikesed kõrvitsad on palju kallimad kui suured, ilmselt seetõttu, et neid mahub bushelisse palju rohkem, arvestades, kui palju ruumi võtab üks suur õõnes pirukakõrvits.
<br>


Nüüd, lihtsalt seikluse pärast 💁‍♀️, liigutame Kuu veeru esimeseks, st `enne` veergu `Package`.

`dplyr::relocate()` kasutatakse veerupositioonide muutmiseks.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Tubli töö!👌 Nüüd on sul puhas ja korrastatud andmestik, mille põhjal saad oma uue regressioonimudeli luua!  
<br>


## 4. Andmete visualiseerimine ggplot2 abil

<p >
   <img src="../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.et.png"
   width="600"/>
   <figcaption>Infograafika autor: Dasani Madipalli</figcaption>


<!--![Infograafika autor: Dasani Madipalli](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.et.png){width="600"}-->

On olemas *tark* ütlus, mis kõlab järgmiselt:

> "Lihtne graafik on andmeanalüütiku mõttemaailma toonud rohkem teavet kui ükski teine vahend." --- John Tukey

Andmeteadlase rolli osaks on näidata andmete kvaliteeti ja olemust, millega nad töötavad. Selleks loovad nad sageli huvitavaid visualiseeringuid, nagu diagrammid, graafikud ja tabelid, mis näitavad andmete erinevaid aspekte. Nii saavad nad visuaalselt esitada seoseid ja lünki, mida muidu oleks raske avastada.

Visualiseeringud aitavad ka kindlaks teha, milline masinõppe meetod on andmete jaoks kõige sobivam. Näiteks hajuvusdiagramm, mis näib järgivat sirget, viitab sellele, et andmed sobivad hästi lineaarse regressiooni harjutuseks.

R pakub mitmeid süsteeme graafikute loomiseks, kuid [`ggplot2`](https://ggplot2.tidyverse.org/index.html) on üks elegantsemaid ja mitmekülgsemaid. `ggplot2` võimaldab graafikuid koostada **sõltumatute komponentide kombineerimise** teel.

Alustame lihtsa hajuvusdiagrammiga, mis kujutab Price (hind) ja Month (kuu) veerge.

Selles näites alustame funktsiooniga [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), määrame andmekogumi ja esteetilise kaardistuse (kasutades [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)), ning lisame kihid (näiteks [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) hajuvusdiagrammi jaoks.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Kas see on kasulik graafik 🤷? Kas miski selles üllatab sind?

See pole eriti kasulik, kuna see lihtsalt kuvab sinu andmed punktide hajutusena konkreetses kuus.
<br>


### **Kuidas muuta see kasulikuks?**

Selleks, et graafikud kuvaksid kasulikku teavet, tuleb andmed tavaliselt kuidagi rühmitada. Näiteks meie puhul annaks kõrvitsate keskmise hinna leidmine iga kuu kohta rohkem ülevaadet andmete aluseks olevatest mustritest. See viib meid veel ühe **dplyr** kiirülevaateni:

#### `dplyr::group_by() %>% summarize()`

R-is saab rühmitatud koondandmeid hõlpsasti arvutada, kasutades

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` muudab analüüsi üksuse kogu andmestikust individuaalseteks rühmadeks, näiteks kuude kaupa.

-   `dplyr::summarize()` loob uue andmeraami, kus igale rühmitamisvariandile on üks veerg ja igale määratud koondstatistikale üks veerg.

Näiteks saame kasutada `dplyr::group_by() %>% summarize()`, et rühmitada kõrvitsad **Month** veeru alusel ja seejärel leida iga kuu kohta **keskmine hind**.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Kategoorilised tunnused, nagu kuud, on paremini esitatavad tulpdiagrammi abil 📊. Tulpdiagrammide loomiseks kasutatakse kihte `geom_bar()` ja `geom_col()`. Lisateabe saamiseks vaata `?geom_bar`.

Teeme ühe valmis!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 See on kasulikum andmete visualiseerimine! Tundub, et kõrvitsate kõrgeim hind esineb septembris ja oktoobris. Kas see vastab sinu ootustele? Miks või miks mitte?

Palju õnne teise õppetunni lõpetamise puhul 👏! Sa valmistasid oma andmed ette mudeli loomiseks ja avastasid seejärel visualiseerimiste abil rohkem teadmisi!



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
